# Data Preprocessing for predicting Transcription Rate (TS)

This notebook is ment to convert raw cell data from several wells into multichannel images (along with its corresponding metadata).

In [1]:
import numpy as np
import pandas as pd
# To display all the columns
pd.options.display.max_columns = None
import os
import sys
import matplotlib.pyplot as plt
import json

# Set paths
BASE_DIR = os.path.realpath(os.path.join(os.path.abspath(''),'../..'))
if not os.path.exists(BASE_DIR):
    print('ERROR!, base path {} does not exist! Setting to None'.format(BASE_DIR))
    BASE_DIR = None
else:
    print('BASE_DIR: {}'.format(BASE_DIR))

DATA_DIR = os.path.join(BASE_DIR, 'datasets', 'raw')
if not os.path.exists(DATA_DIR):
    print('ERROR!, data path {} does not exist! Setting to None'.format(DATA_DIR))
    DATA_DIR = None
else:
    print('DATA_DIR: {}\n'.format(DATA_DIR))
    
# Add BASE_DIR to sys paths (for loading libraries)
sys.path.insert(1, os.path.join(BASE_DIR, 'workspace'))
# Load mpp_data library to convert raw data into images
from pelkmans.mpp_data import MPPData as MPPData

# List available local Wells
wells = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
print('Available local wells: \n', wells)


BASE_DIR: /home/hhughes/Documents/Master_Thesis/Project
DATA_DIR: /home/hhughes/Documents/Master_Thesis/Project/datasets/raw

Setting BASE_DIR to /home/hhughes/Documents/Master_Thesis/Project
Available local wells: 
 ['I11', 'I09', 'J10']


Set parameters for data transformation:

In [2]:
# In case you only want to load some specific wells, rename 'wells'
wells = ['I09']

data_params = {
    # where to read data from
    'data_dirs': [os.path.join(DATA_DIR, well) for well in wells],
    'dir_type': 'hannah',
    # make results reproducible
    'seed': 42,
    # input/output definition
    'input_channels': [
        '00_DAPI',
        '07_H2B',
        '01_CDK9_pT186',
        '03_CDK9',
        '05_GTF2B',
        '07_SETD1A',
        '08_H3K4me3',
        '09_SRRM2',
        '10_H3K27ac',
        '11_KPNA2_MAX',
        '12_RB1_pS807_S811',
        '13_PABPN1',
        '14_PCNA',
        '15_SON',
        '16_H3',
        '17_HDAC3',
        '19_KPNA1_MAX',
        '20_SP100',
        '21_NCL',
        '01_PABPC1',
        '02_CDK7',
        '03_RPS6',
        '05_Sm',
        '07_POLR2A',
        '09_CCNT1',
        '10_POL2RA_pS2',
        '11_PML',
        '12_YAP1',
        '13_POL2RA_pS5',
        '15_U2SNRNPB',
        '18_NONO',
        '20_ALYREF',
        '21_COIL',
    ],
    'output_channels': ['00_EU'],
    'aggregate_output': 'avg', # None results in output images, 'max', 'avg' aggregate output channels and output a single number
    # train/val/test split
    'train_frac': 0.8,
    'val_frac': 0.1,
    'img_size': 224,
    # normalisation
    'background_value': os.path.join(DATA_DIR, 'secondary_only_relative_normalisation.csv'),
    'normalise': True,
    'percentile': 98.0,
    # Add Cell cycle to metadata
    'add_cell_cycle_to_metadata': True,
    'cell_cycle_file': os.path.join(DATA_DIR, 'cell_cycle_classification.csv'),
    # Add well info to metadata (cell_type, perturbation and duration)
    'add_well_info_to_metadata': True,
    'well_info_file': os.path.join(DATA_DIR, 'wells_metadata.csv'),
    # Fitering
    'filter_criteria': ['is_border_cell', 'is_mitotic'],
    'filter_values': [0, 0],
}

Process data:

In [3]:
p = data_params

mpp_datas = {'train': [], 'val': [], 'test': []}
for data_dir in p['data_dirs']:
    print('\nProcessing Well {}...'.format(data_dir))
    # Load data as an MPPData object
    mpp_data = MPPData.from_data_dir(data_dir,
                                     dir_type=p['dir_type'],
                                     seed=p['seed'])
    
    # Remove unwanted cells
    if p.get('filter_criteria', None) is not None:
        print('removing unwanted cells...')
        mpp_data.filter_cells(p['filter_criteria'], p['filter_values'])

    # Subtract background  values for each channel
    if p['normalise']:
        mpp_data.subtract_background(p['background_value'])
        
    
    # Split data into train, validation and test
    train, val, test = mpp_data.train_val_test_split(p['train_frac'], p['val_frac'])
    
    # Save well data in dic
    mpp_datas['train'].append(train)
    mpp_datas['test'].append(test)
    mpp_datas['val'].append(val)
    
    # Release memory
    del(train)
    del(val)
    del(test)
    del(mpp_data)
    
# merge data from all the loaded wells
train = MPPData.concat(mpp_datas['train'])
val = MPPData.concat(mpp_datas['val'])
test = MPPData.concat(mpp_datas['test'])
del(mpp_datas)

# Normalize train, val and test using the normalization parameters
# got from the train data (inner percentile% of train data)
if p['normalise']:
    rescale_values = train.rescale_intensities_per_channel(percentile=p['percentile'])
    _ = val.rescale_intensities_per_channel(rescale_values=rescale_values)
    _ = test.rescale_intensities_per_channel(rescale_values=rescale_values)
    p['normalise_rescale_values'] = list(rescale_values)

# Add cell cycle to metadata (G1, S, G2)
if p['add_cell_cycle_to_metadata']:
    train.add_cell_cycle_to_metadata(p['cell_cycle_file'])
    val.add_cell_cycle_to_metadata(p['cell_cycle_file'])
    test.add_cell_cycle_to_metadata(p['cell_cycle_file'])

if p['add_well_info_to_metadata']:
    train.add_well_info_to_metadata(p['well_info_file'])
    val.add_well_info_to_metadata(p['well_info_file'])
    test.add_well_info_to_metadata(p['well_info_file'])


Processing Well /home/hhughes/Documents/Master_Thesis/Project/datasets/raw/I09...
removing unwanted cells...
Total number of cells: 886

269 cells cutted by filter: is_border_cell != 0
48 cells cutted by filter: is_mitotic != 0

Final number of cells cutted: 304


missing background value for channels ['00_EU', '09_SRRM2_ILASTIK', '15_SON_ILASTIK']


In [4]:
train.metadata

mapobject_id plate_name well_name  well_pos_y  well_pos_x  tpoint  \
0          373535    plate01       I09           0           0       0   
1          373536    plate01       I09           0           0       0   
2          373537    plate01       I09           0           0       0   
3          373538    plate01       I09           0           0       0   
4          373539    plate01       I09           0           0       0   
..            ...        ...       ...         ...         ...     ...   
460        248097    plate01       I09           5           4       0   
461        248098    plate01       I09           5           4       0   
462        248100    plate01       I09           5           4       0   
463        248102    plate01       I09           5           4       0   
464        248109    plate01       I09           5           4       0   

     zplane  label  is_border  mapobject_id_cell plate_name_cell  \
0         0      6          0             373506         plate01   
1         0      7          0             373507         plate01   
2         0      8          0             373508         plate01   
3         0      9          0             373509         plate01   
4         0     10          0             373510         plate01   
..      ...    ...        ...                ...             ...   
460       0     32          0             248052         plate01   
461       0     33          0             248053         plate01   
462       0     35          0             248055         plate01   
463       0     37          0             248057         plate01   
464       0     44          0             248064         plate01   

    well_name_cell  well_pos_y_cell  well_pos_x_cell  tpoint_cell  \
0              I09                0                0            0   
1              I09                0                0            0   
2              I09                0                0            0   
3              I09                0                0            0   
4              I09                0                0            0   
..             ...              ...              ...          ...   
460            I09                5                4            0   
461            I09                5                4            0   
462            I09                5                4            0   
463            I09                5                4            0   
464            I09                5                4            0   

     zplane_cell  label_cell  is_border_cell  is_mitotic  is_mitotic_labels  \
0              0           6               0         0.0                NaN   
1              0           7               0         0.0                NaN   
2              0           8               0         0.0                NaN   
3              0           9               0         0.0                NaN   
4              0          10               0         0.0                NaN   
..           ...         ...             ...         ...                ...   
460            0          32               0         0.0                NaN   
461            0          33               0         0.0                NaN   
462            0          35               0         0.0                NaN   
463            0          37               0         0.0                NaN   
464            0          44               0         0.0                NaN   

     is_polynuclei_HeLa  is_polynuclei_HeLa_labels  is_polynuclei_184A1  \
0                   0.0                        NaN                  0.0   
1                   0.0                        NaN                  0.0   
2                   0.0                        NaN                  0.0   
3                   0.0                        NaN                  0.0   
4                   0.0                        NaN                  0.0   
..                  ...                        ...                  ...

In [5]:
val.metadata

mapobject_id plate_name well_name  well_pos_y  well_pos_x  tpoint  zplane  \
0         373548    plate01       I09           0           0       0       0   
1         373550    plate01       I09           0           0       0       0   
2         228068    plate01       I09           0           1       0       0   
3         324142    plate01       I09           0           2       0       0   
4         291040    plate01       I09           0           3       0       0   
5         291044    plate01       I09           0           3       0       0   
6         291048    plate01       I09           0           3       0       0   
7         291056    plate01       I09           0           3       0       0   
8         383784    plate01       I09           0           4       0       0   
9         383785    plate01       I09           0           4       0       0   
10        383804    plate01       I09           0           4       0       0   
11        332647    plate01       I09           1           0       0       0   
12        332648    plate01       I09           1           0       0       0   
13        345899    plate01       I09           1           4       0       0   
14        345919    plate01       I09           1           4       0       0   
15        271267    plate01       I09           2           0       0       0   
16        271283    plate01       I09           2           0       0       0   
17        271284    plate01       I09           2           0       0       0   
18        271298    plate01       I09           2           0       0       0   
19        271303    plate01       I09           2           0       0       0   
20        271304    plate01       I09           2           0       0       0   
21        205773    plate01       I09           2           1       0       0   
22        205779    plate01       I09           2           1       0       0   
23        205790    plate01       I09           2           1       0       0   
24        205791    plate01       I09           2           1       0       0   
25        306933    plate01       I09           2           2       0       0   
26        256555    plate01       I09           2           3       0       0   
27        256562    plate01       I09           2           3       0       0   
28        256563    plate01       I09           2           3       0       0   
29        256566    plate01       I09           2           3       0       0   
30        359174    plate01       I09           3           0       0       0   
31        359187    plate01       I09           3           0       0       0   
32        359370    plate01       I09           3           1       0       0   
33        359381    plate01       I09           3           1       0       0   
34        280801    plate01       I09           3           2       0       0   
35        280805    plate01       I09           3           2       0       0   
36        280832    plate01       I09           3           2       0       0   
37        259762    plate01       I09           3           3       0       0   
38        259769    plate01       I09           3           3       0       0   
39        259771    plate01       I09           3           3       0       0   
40        259781    plate01       I09           3           3       0       0   
41        331839    plate01       I09           4           0       0       0   
42        366472    plate01       I09           4           2       0       0   
43        366494    plate01       I09           4           2       0       0   
44        366506    plate01       I09           4           2       0       0   
45        200764    plate01       I09           4           3       0       0   
46        200770    plate01       I09           4           3       0       0   
47        200781    plate01       I09           4           3       0       0   
48        347714    plate01       I

In [6]:
test.metadata

mapobject_id plate_name well_name  well_pos_y  well_pos_x  tpoint  zplane  \
0         228079    plate01       I09           0           1       0       0   
1         324146    plate01       I09           0           2       0       0   
2         291066    plate01       I09           0           3       0       0   
3         291073    plate01       I09           0           3       0       0   
4         291075    plate01       I09           0           3       0       0   
5         383787    plate01       I09           0           4       0       0   
6         383800    plate01       I09           0           4       0       0   
7         383312    plate01       I09           1           3       0       0   
8         345912    plate01       I09           1           4       0       0   
9         271270    plate01       I09           2           0       0       0   
10        271272    plate01       I09           2           0       0       0   
11        271288    plate01       I09           2           0       0       0   
12        271300    plate01       I09           2           0       0       0   
13        205787    plate01       I09           2           1       0       0   
14        256547    plate01       I09           2           3       0       0   
15        256557    plate01       I09           2           3       0       0   
16        231603    plate01       I09           2           4       0       0   
17        231612    plate01       I09           2           4       0       0   
18        231617    plate01       I09           2           4       0       0   
19        359185    plate01       I09           3           0       0       0   
20        359190    plate01       I09           3           0       0       0   
21        359191    plate01       I09           3           0       0       0   
22        359376    plate01       I09           3           1       0       0   
23        359387    plate01       I09           3           1       0       0   
24        359388    plate01       I09           3           1       0       0   
25        359389    plate01       I09           3           1       0       0   
26        359399    plate01       I09           3           1       0       0   
27        280817    plate01       I09           3           2       0       0   
28        280822    plate01       I09           3           2       0       0   
29        280834    plate01       I09           3           2       0       0   
30        259768    plate01       I09           3           3       0       0   
31        259775    plate01       I09           3           3       0       0   
32        356246    plate01       I09           3           4       0       0   
33        331848    plate01       I09           4           0       0       0   
34        331854    plate01       I09           4           0       0       0   
35        366470    plate01       I09           4           2       0       0   
36        366489    plate01       I09           4           2       0       0   
37        366495    plate01       I09           4           2       0       0   
38        366497    plate01       I09           4           2       0       0   
39        200753    plate01       I09           4           3       0       0   
40        200755    plate01       I09           4           3       0       0   
41        200772    plate01       I09           4           3       0       0   
42        347710    plate01       I09           4           4       0       0   
43        347725    plate01       I09           4           4       0       0   
44        337121    plate01       I09           5           0       0       0   
45        337123    plate01       I09           5           0       0       0   
46        337125    plate01       I09           5           0       0       0   
47        328150    plate01       I09           5           1       0       0   
48        328156    plate01       I

## Save data

Prepare to save data:

In [7]:
import shutil

# create dir
dataset_name = '184A1_hannah_EU_regression'
outdir = os.path.join(BASE_DIR, 'datasets', dataset_name)
if os.path.exists(outdir):
    print('Warning! Directory {} already exist! Deleting...\n'.format(outdir))
    try:
        shutil.rmtree(outdir)
    except OSError as e:
        print('Dir {} could not be deleted!\n\nOSError: {}'.format(outdir, e))

print('Creating dir: {}'.format(outdir))
os.makedirs(outdir, exist_ok=False)
    

Warning! Directory /home/hhughes/Documents/Master_Thesis/Project/datasets/184A1_hannah_EU_regression already exist! Deleting...

Creating dir: /home/hhughes/Documents/Master_Thesis/Project/datasets/184A1_hannah_EU_regression


In [8]:
# Get channels ids (proteins) which will be used to predict transcripcion rate
input_ids = list(train.channels.set_index('name').loc[p['input_channels']]['channel_id'])
# Get id of the channel that measure trancripcion rate
output_ids = list(train.channels.set_index('name').loc[p['output_channels']]['channel_id'])
# add output channel id after the input channels ids
ids = input_ids + output_ids

Save metadata and used parameters

In [9]:
# save params
json.dump(data_params, open(os.path.join(outdir, 'params.json'), 'w'), indent=4)

# save metadata
train.metadata.to_csv(os.path.join(outdir, 'train_metadata.csv'))
val.metadata.to_csv(os.path.join(outdir, 'val_metadata.csv'))
test.metadata.to_csv(os.path.join(outdir, 'test_metadata.csv'))
pd.concat([train.metadata, val.metadata, test.metadata]).to_csv(os.path.join(outdir, 'metadata.csv'))

# Save used channels
#train.channels.to_csv(os.path.join(outdir, 'channels.csv'))
train.channels.set_index('channel_id').loc[ids].to_csv(os.path.join(outdir, 'channels.csv'))

Save Images

In [10]:
# Note! instead of calculating the response value (y) here and save
# it separatelly, instead we will do it on the modeling part

"""
# get images
train_dataset = np.array(train.get_object_imgs(data='MPP', img_size=p['img_size']))
del(train)
val_dataset = np.array(val.get_object_imgs(data='MPP', img_size=p['img_size']))
del(val)
test_dataset = np.array(test.get_object_imgs(data='MPP', img_size=p['img_size']))
del(test)

# Create responce variable (y)
if p['aggregate_output'] == 'avg':
    train_dataset_y = np.array([img[img!=0].mean() for img in train_dataset[:,:,:,output_ids]])
    val_dataset_y = np.array([img[img!=0].mean() for img in val_dataset[:,:,:,output_ids]])
    test_dataset_y = np.array([img[img!=0].mean() for img in test_dataset[:,:,:,output_ids]])

# Save datasets
np.savez(os.path.join(outdir, 'train_dataset.npz'), x=train_dataset[:,:,:,input_ids], y=train_dataset_y)
del(train_dataset)
np.savez(os.path.join(outdir, 'val_dataset.npz'), x=val_dataset[:,:,:,input_ids], y=val_dataset_y)
del(val_dataset)
np.savez(os.path.join(outdir, 'test_dataset.npz'), x=test_dataset[:,:,:,input_ids], y=test_dataset_y)
del(test_dataset)
"""

# get images, save them and delete vars
train_dataset = np.array(train.get_object_imgs(data='MPP', img_size=p['img_size']))
np.save(os.path.join(outdir, 'train_dataset.npy'), train_dataset[:,:,:,ids])
del(train_dataset)
del(train)

val_dataset = np.array(val.get_object_imgs(data='MPP', img_size=p['img_size']))
np.save(os.path.join(outdir, 'val_dataset.npy'), val_dataset[:,:,:,ids])
del(val_dataset)
del(val)

test_dataset = np.array(test.get_object_imgs(data='MPP', img_size=p['img_size']))
np.save(os.path.join(outdir, 'test_dataset.npy'), test_dataset[:,:,:,ids])
del(test_dataset)
del(test)